In [1]:
%matplotlib widget

import matplotlib.pyplot as plt
plt.ioff()

from smfret_analysis import Tracker, print_info

In [2]:
# Create a Tracker instance

# If starting fresh, set load = False. If there is already saved data from a previous run,
# load = True can be used to load it.
load = False

if not load:
    # Directory containing raw data
    data_dir = "sm-data"
    
    # Specify
    # - excitation sequence, where "c" means cell contours, "a" means acceptor excitation,
    #   and "d" means donor excitation
    # - data directory
    tr = Tracker("se + da * 300 + s", data_dir=data_dir)
    
    # Add datasets by giving
    # - an identifier
    # - a regular expression describing the file names relative to data_dir.
    #   Use forward slashes as path separators even on Windows.
    tr.add_dataset("DPPC control cells",
                   r"^DPPC_ctrl-J4/cells.*\.tif")
    tr.add_dataset("DPPC control no-cells",
                   r"^DPPC_ctrl-J4/no-cells.*\.tif")
    # If donor and acceptor emission channels are recorded by separate cameras, resulting
    # in separate video files, the second argument is a regular expression for the donor,
    # the third argument a regular expression for the acceptor emission. First donor file
    # will be matched with first acceptor file, second with second, etc., so make sure
    # they have the same sorting order.
    # tr.add_dataset("DPPC control cells",
    #                r"^DPPC_ctrl-J4/cells.*_donor\.tif",
    #                r"^DPPC_ctrl-J4/cells.*_acceptor\.tif")
    # tr.add_dataset("DPPC control no-cells",
    #                r"^DPPC_ctrl-J4/no-cells.*_donor\.tif",
    #                r"^DPPC_ctrl-J4/no-cells.*_acceptor\.tif")

    # Special datasets for registration etc. If donor and acceptor emission are recorded
    # in separate files, again use two regular expressions.
    tr.add_special_dataset("registration", r"beads/beads_.*\.tif")
    tr.add_special_dataset("donor-profile", "profile/green_.*\.tif")
    tr.add_special_dataset("acceptor-profile", "profile/red_.*\.tif")    
    # tr.add_special_dataset("donor-only", r"^DO/no-cells.*\.tif")
    # tr.add_special_dataset("acceptor-only", r"^AO/no-cells.*\.tif")
else:
    # Load
    tr = Tracker.load()

In [3]:
# Donor and acceptor emission channels were recorded side-by-side on the same
# camera. Use the UI to define where the channels are. To do so, load some
# exemplary files matching the reg. ex. passed to the method call.
tr.split_channels(r"beads/beads.*\.tif")

_ChannelSplitter(children=(ImageSelector(children=(Button(disabled=True, icon='arrow-left', layout=Layout(disp…

In [4]:
# Set localization options for fiducial markers, which are used below to
# calculate the coordinate transform between donor and acceptor channels.
tr.set_registration_loc_opts()

In [5]:
# Calculate the coordinate transform from bead localizations
tr.calc_registration()

Label(value='Starting…')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
tr.set_loc_opts()

In [7]:
# Run localization algorithm. This can take a while.
tr.locate()

Label(value='Starting…')

In [8]:
# Track the localizations. 
tr.track(feat_radius=4, bg_frame=3, link_radius=5, link_mem=3, min_length=4, bg_estimator="mean")

Label(value='Starting…')

In [9]:
# Get the images of the cells
tr.extract_segment_images()

# Get the laser profiles from additional recordings of bulk samples
tr.make_flatfield("donor", bg=200, smooth_sigma=10)
tr.make_flatfield("acceptor", bg=200, smooth_sigma=10)

In [10]:
# Save data
tr.save()